<a href="https://colab.research.google.com/github/AgomferAustral/DMA-Caras/blob/main/deteccionaumentadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deteccion y recorte de caras - Fase 2
### Version 1.0
## Origenes: Detectadas y Aumentadas
## Destinos: Detectadas2 y Aumentadas2
### Recorta el marco de la cara, no la cabeza completa

In [ ]:
import os 

if not os.path.exists('/content/drive/MyDrive'):
    from google.colab import drive
    drive.mount('/content/drive')

Cargar las librerias que vamos a utilizar

In [ ]:
import sys
import subprocess

def instalar_si_falta(paquete, nombre_import=None):
    if nombre_import is None:
        nombre_import = paquete
    try:
        __import__(nombre_import)
    except ImportError:
        print(f"Instalando {paquete}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", paquete])

# Lista de paquetes a instalar/importar
paquetes = [
    ("cv2", "opencv-python"),
    ("math", "math"),
    ("numpy", "numpy"),
    ("PIL", "Pillow"),
]

for nombre_import, paquete in paquetes:
    instalar_si_falta(paquete, nombre_import)

# Ahora puedes importar normalmente
import cv2
import math
import numpy as np
from sklearn.manifold import Isomap
from PIL import Image


In [ ]:
# Montar Google Drive si es necesario
if not os.path.exists('/content/drive/MyDrive'):
    from google.colab import drive
    drive.mount('/content/drive')

# Verificar si la carpeta de salida existe, si no, crearla
img_size = (80, 80)
ruta_entrada = '/content/drive/MyDrive/Detectadas'
ruta_salida = '/content/drive/MyDrive/Detectadas2'

if not os.path.exists(ruta_salida):
    os.makedirs(ruta_salida)

# Inicializar el clasificador de rostros de OpenCV
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def procesar_archivos_en_carpetas(ruta_principal, ruta_final, imgsize):
    """
    Recorre las carpetas dentro de la ruta principal y procesa los archivos.
    Args:
        ruta_principal: La ruta de la carpeta principal.
        ruta_final: La ruta de la carpeta de salida.
        imgsize: Tupla con el tamaño de la imagen de salida.
    """
    fraccion = 8
    x_inicial = imgsize[0] // (fraccion - 2)
    y_inicial = imgsize[1] // (fraccion - 2)
    alto = imgsize[1]
    ancho = imgsize[0]
    multip = fraccion / (fraccion - 2)
    img_size = (math.floor(ancho * multip), math.floor(alto * multip))

    print(f"Ruta principal: {ruta_principal}")
    for carpeta_actual, _, archivos in os.walk(ruta_principal):
        print(f"Carpeta actual: {carpeta_actual}")
        for archivo in archivos:
            print(f"Archivo: {archivo}")
            ruta_completa = os.path.join(carpeta_actual, archivo)
            try:
                img = cv2.imread(ruta_completa, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"⚠️ No se pudo leer: {ruta_completa}")
                    continue

                img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

                # Detectar rostros
                faces = face_cascade.detectMultiScale(
                    img,
                    scaleFactor=1.1,
                    minNeighbors=10,
                    minSize=(80, 80)
                )

                for i, (x, y, w, h) in enumerate(faces):
                    face = img[y:y + h, x:x + w]  # Recortar rostro
                    face_resized = cv2.resize(face, img_size)  # Redimensionar
                    face_cutted = face_resized[
                        y_inicial:y_inicial + alto,
                        x_inicial:x_inicial + ancho
                    ]

                    # Crear carpeta de salida manteniendo la estructura original
                    relative_path = os.path.relpath(carpeta_actual, ruta_entrada)
                    output_folder = os.path.join(ruta_salida, relative_path)
                    os.makedirs(output_folder, exist_ok=True)

                    # Guardar rostro detectado
                    output_path = os.path.join(
                        output_folder,
                        f"{os.path.splitext(archivo)[0]}_face{i}.jpg"
                    )
                    cv2.imwrite(output_path, face_cutted)
                    print(f"✅ Guardado: {output_path}")

            except Exception as e:
                print(f"❌ Error procesando {ruta_completa}: {e}")

print("Detectar Caras ...")
procesar_archivos_en_carpetas(ruta_entrada, ruta_salida, img_size)
print("Detección terminada ...")
print("Para entrenamiento, hay que limpiar manualmente la carpeta de salida, para eliminar los recortes que no son rostros.")

Detectar Caras ...
Ruta principal: /content/drive/MyDrive/Detectadas
Carpeta actual: /content/drive/MyDrive/Detectadas
Carpeta actual: /content/drive/MyDrive/Detectadas/Federico
Archivo: IMG_1564_face0.jpg
✅ Guardado: /content/drive/MyDrive/Detectadas2/Federico/IMG_1564_face0_face0.jpg
Archivo: 1742907850028_face0.jpg
✅ Guardado: /content/drive/MyDrive/Detectadas2/Federico/1742907850028_face0_face0.jpg
Archivo: IMG_1562_face0.jpg
✅ Guardado: /content/drive/MyDrive/Detectadas2/Federico/IMG_1562_face0_face0.jpg
Archivo: IMG_1568_face0.jpg
✅ Guardado: /content/drive/MyDrive/Detectadas2/Federico/IMG_1568_face0_face0.jpg
Archivo: IMG_1189_face0.jpg
✅ Guardado: /content/drive/MyDrive/Detectadas2/Federico/IMG_1189_face0_face0.jpg
Archivo: IMG_1192_face0.jpg
Archivo: 20250321_140311_face0.jpg
✅ Guardado: /content/drive/MyDrive/Detectadas2/Federico/20250321_140311_face0_face0.jpg
Archivo: IMG_1078_face0.jpg
Archivo: IMG_1079_face0.jpg
✅ Guardado: /content/drive/MyDrive/Detectadas2/Federico/IMG_

In [ ]:

# Montar Google Drive si es necesario
if not os.path.exists('/content/drive/MyDrive'):
    from google.colab import drive
    drive.mount('/content/drive')

# Verificar si la carpeta de salida existe, si no, crearla
img_size = (80, 80)
ruta_entrada = '/content/drive/MyDrive/Aumentadas'
ruta_salida = '/content/drive/MyDrive/Aumentadas2'

if not os.path.exists(ruta_salida):
    os.makedirs(ruta_salida)

# Inicializar el clasificador de rostros de OpenCV
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def procesar_archivos_en_carpetas(ruta_principal, ruta_final, imgsize):
    """
    Recorre las carpetas dentro de la ruta principal y procesa los archivos.
    Args:
        ruta_principal: La ruta de la carpeta principal.
        ruta_final: La ruta de la carpeta de salida.
        imgsize: Tupla con el tamaño de la imagen de salida.
    """
    fraccion = 8
    x_inicial = imgsize[0] // (fraccion - 2)
    y_inicial = imgsize[1] // (fraccion - 2)
    alto = imgsize[1]
    ancho = imgsize[0]
    multip = fraccion / (fraccion - 2)
    img_size = (math.floor(ancho * multip), math.floor(alto * multip))

    print(f"Ruta principal: {ruta_principal}")
    for carpeta_actual, _, archivos in os.walk(ruta_principal):
        print(f"Carpeta actual: {carpeta_actual}")
        for archivo in archivos:
            print(f"Archivo: {archivo}")
            ruta_completa = os.path.join(carpeta_actual, archivo)
            try:
                img = cv2.imread(ruta_completa, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"⚠️ No se pudo leer: {ruta_completa}")
                    continue

                img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

                # Detectar rostros
                faces = face_cascade.detectMultiScale(
                    img,
                    scaleFactor=1.1,
                    minNeighbors=10,
                    minSize=(80, 80)
                )

                for i, (x, y, w, h) in enumerate(faces):
                    face = img[y:y + h, x:x + w]  # Recortar rostro
                    face_resized = cv2.resize(face, img_size)  # Redimensionar
                    face_cutted = face_resized[
                        y_inicial:y_inicial + alto,
                        x_inicial:x_inicial + ancho
                    ]

                    # Crear carpeta de salida manteniendo la estructura original
                    relative_path = os.path.relpath(carpeta_actual, ruta_entrada)
                    output_folder = os.path.join(ruta_salida, relative_path)
                    os.makedirs(output_folder, exist_ok=True)

                    # Guardar rostro detectado
                    output_path = os.path.join(
                        output_folder,
                        f"{os.path.splitext(archivo)[0]}_face{i}.jpg"
                    )
                    cv2.imwrite(output_path, face_cutted)
                    print(f"✅ Guardado: {output_path}")

            except Exception as e:
                print(f"❌ Error procesando {ruta_completa}: {e}")

print("Detectar Caras ...")
procesar_archivos_en_carpetas(ruta_entrada, ruta_salida, img_size)
print("Detección terminada ...")
print("Para entrenamiento, hay que limpiar manualmente la carpeta de salida, para eliminar los recortes que no son rostros.")

Se han truncado las últimas 5000 líneas del flujo de salida.
Archivo: IMG_4168_face0_rot10.png
✅ Guardado: /content/drive/MyDrive/Aumentadas2/Daniel/IMG_4168_face0_rot10_face0.jpg
Archivo: IMG_20250321_140511746_face0_dx5pct.png
✅ Guardado: /content/drive/MyDrive/Aumentadas2/Daniel/IMG_20250321_140511746_face0_dx5pct_face0.jpg
Archivo: IMG_4168_face0_dy-5pct.png
✅ Guardado: /content/drive/MyDrive/Aumentadas2/Daniel/IMG_4168_face0_dy-5pct_face0.jpg
Archivo: IMG_4168_face0_rot-10.png
✅ Guardado: /content/drive/MyDrive/Aumentadas2/Daniel/IMG_4168_face0_rot-10_face0.jpg
Archivo: IMG_20250321_140511746_face0_dy-5pct.png
✅ Guardado: /content/drive/MyDrive/Aumentadas2/Daniel/IMG_20250321_140511746_face0_dy-5pct_face0.jpg
Archivo: 20250321_140850_face0_dy5pct.png
✅ Guardado: /content/drive/MyDrive/Aumentadas2/Daniel/20250321_140850_face0_dy5pct_face0.jpg
Archivo: 20250321_140850_face0_dx5pct.png
✅ Guardado: /content/drive/MyDrive/Aumentadas2/Daniel/20250321_140850_face0_dx5pct_face0.jpg
Archiv